## *Long short-term memory* (Memória de curto prazo)

# Agora vamos resolver um problema prático e bastante desafiador, para verificar o real poder das redes neurais artificiais. Como de costume, vamos primeiro investigar a base de dados que utilizaremos. Dê uma olhada:
  
https://github.com/Natalnet/GCiD/raw/master/Codes/Data/curitibadiario.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Leitura de dados
dataSet = pd.read_csv("https://github.com/Natalnet/GCiD/raw/master/Codes/Data/curitibadiario.csv",
                      skiprows = 16, sep=';')


In [ ]:
#Visualizando cabeçalho dos dados
dataSet.head()

# Velho problema conhecido, que já sabemos como resolver. Vamos simplificar e juntar todos os passos (já estudados) em uma função, que vai retornar nossa base de dados bem mais organizada.

In [ ]:
def OrganizarDados(dataSet):
    #Transformando data em variável do tipo datetime
    dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")
    #Atribuindo "Data" como índice para a base de dados
    dataSet = dataSet.set_index("Data")
    #Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
    dataSet00, dataSet12 = dataSet[dataSet["Hora"] == 0], dataSet[dataSet["Hora"] == 1200]
    #Descartando as colunas "Hora", "Estacao" e "Unnamed: 11" 
    dataSet00, dataSet12 = dataSet00.drop(columns = ["Hora", "Estacao", "Unnamed: 11"]), dataSet12.drop(columns = ["Hora", "Estacao", "Unnamed: 11"])
    #Eliminando colunas completas por "NaN" em cada uma das bases de dados
    dataSet00 = dataSet00.drop(columns = ["Precipitacao","TempMinima"])
    dataSet12 = dataSet12.drop(columns = ["TempMaxima","Insolacao","Evaporacao Piche","Temp Comp Media","Umidade Relativa Media","Velocidade do Vento Media"])
    #Criando o intervalo completo de tempo de 01-01-2005 à 31-12-2017
    dataInicial = '2005-01-01'
    dataFinal = '2017-12-31'
    tempo = pd.date_range(dataInicial, dataFinal)
    #Atribuindo este intervalo de tempo à um dataSet provisório
    dataSetProv = pd.DataFrame()
    dataSetProv["Data"] = tempo
    #Atribuindo o índice para o dataSet provisório como sendo a coluna de datas
    dataSetProv = dataSetProv.set_index("Data")
    #Mesclando o dataSet provisório como sendo o resultado da junção dos dataSet00 e dataSet12
    dataSetProv = dataSetProv.join(dataSet00).join(dataSet12)
    #Tornando o dataSet como sendo o dataSet provisório
    dataSet = dataSetProv  
    return dataSet

In [ ]:
#Aplicando a função ao dataSet
dataSet = OrganizarDados(dataSet)

# Nosso objetivo hoje é prever temperatura máxima para um determinado dia, dada uma determinada combinação de outras variáveis. Será que conseguimos?

In [ ]:
#Visualizando relação dos dados
plt.scatter(dataSet.index, dataSet["TempMaxima"])
plt.title("Distribuição de temperatura máxima ao longo dos últimos anos")
plt.xlabel("Data")
plt.ylabel("Temperatura máxima diária")
plt.show()

# Não parece nada simples, hein? Desafio você a tentar modelar uma função polinomial que descreva o comportamento desse problema. O último ser humano que tentou realizar essa proeza foi encontrado meses depois congelado em uma caverna no Alaska (fonte: FEI - Fatos que Eu Inventei)

In [ ]:
#Eliminando eventos inconsistentes da base de dados
dataSet = dataSet.dropna()

In [ ]:
#Resentando o índice do dataSet
dataSet = dataSet.reset_index(drop = True)

In [ ]:
#Visualizando cabeçalho dos dados
dataSet.head()

In [ ]:
#Visualizando descrição resumida dos dados
dataSet.describe()

In [ ]:
#Importando biblioteca útil
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Separando "TempMaxima" de dataSet
tempMax = dataSet["TempMaxima"].values

In [ ]:
tempMax[0:11]

In [ ]:
#Preparando dados para a LSTM
x_train, y_train = [], []
for i in range(10, len(tempMax)):
    x_train.append(tempMax[(i-10):i])
    y_train.append(tempMax[i])

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
#Efetuando uma transformação linear entre o mínimo e o máximo, linearizando entre (-1,1) para todas as variáveis
mms = MinMaxScaler(feature_range=(-1,1))
tempMax = mms.fit_transform(tempMax.reshape(-1,1))

In [ ]:
#Preparando dados para a LSTM
x_train, y_train = [], []
for i in range(10, len(tempMax)):
    x_train.append(tempMax[(i-10):i])
    y_train.append(tempMax[i])

In [ ]:
#Transformando x_train e y_train em numpyArray
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Dimensões da base de dados de entrada para treino
x_train.shape

In [ ]:
#Dimensões da base de dados de saída para treino
y_train.shape

In [ ]:
#Substituindo NaN por 0
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [ ]:
#Importando bibliotecas para treinar LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
#Criando o modelo de previsão do tipo sequential
regressor = Sequential()

In [ ]:
#Adicionando camada de entrada
regressor.add(LSTM(units = 10, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))

In [ ]:
#Adicionando primeira camada escondida
regressor.add(LSTM(units = 10, return_sequences = True))

In [ ]:
#Adicionando segunda camada escondida
regressor.add(LSTM(units = 10))

In [ ]:
#Adicionando camada de saída
regressor.add(Dense(activation = "tanh", units = y_train.shape[1]))

In [ ]:
#Compilando a rede neural
regressor.compile(optimizer = "adam", loss = "mean_squared_error")

In [ ]:
#Ajustando a rede neural
regressor.fit(x_train, y_train, epochs = 50, batch_size = 32)

In [ ]:
#Previsão
y_predicted = regressor.predict(x_train)

In [ ]:
#Bibliotecas para validar o funcionamento da rede
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt